In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
import os
import json 
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from datetime import datetime
from keras.applications.vgg16 import preprocess_input 

SEED = 1234
tf.random.set_seed(SEED)  

# Get current working directory
cwd = os.getcwd()

In [ ]:
def create_csv(results, results_dir='/content/drive/My Drive/DeepLearning'):

    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'

    with open(os.path.join(results_dir, csv_fname), 'w') as f:

        f.write('Id,Category\n')

        for key, value in results.items():
            f.write(key + ',' + str(value) + '\n')
            print("writing "+ key + ',' + str(value) + '\n')


In [ ]:
# Run this cell only if you are using Colab with Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Run this cell only if you are using Colab with Drive
!unzip '/content/drive/My Drive/DeepLearning/kaggle/data/artificial-neural-networks-and-deep-learning-2020.zip'

In [ ]:
labels_dir = os.path.join(cwd, 'MaskDataset')
training_dir = os.path.join(cwd, 'MaskDataset/training')

bs = 8 
img_h = img_w = 512

with open(os.path.join(labels_dir, "train_gt.json")) as f:
  dic = json.load(f)
labels = pd.DataFrame(dic.items())
labels.rename(columns = {0:'filename', 1:'class'}, inplace = True)
labels["class"] = labels["class"].astype(str)

# Create training ImageDataGenerator object


tl, vl = train_test_split(labels, test_size=0.20, random_state=SEED)

tl.values[0]

apply_data_augmentation = True

if apply_data_augmentation: #uscivano immagini a pallini con data aug ora colorate a causa di preproc da solo
    train_data_gen = ImageDataGenerator(rotation_range=30,
                                        width_shift_range=5,
                                        height_shift_range=5,
                                        zoom_range=0.2,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        fill_mode='constant',
                                        cval=0,
                                        rescale=1./255                                     
                                        )
else :   
    train_data_gen = ImageDataGenerator(rescale=1./255)

valid_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_generator = train_data_gen.flow_from_dataframe(tl,
                                               training_dir,
                                               batch_size=bs,
                                               color_mode='rgb',
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED,
                                               target_size=(img_h, img_w)
                                               )

valid_generator = valid_gen.flow_from_dataframe(vl,training_dir,batch_size=bs, 
                                               class_mode='categorical',
                                               color_mode='rgb',
                                               shuffle=False,
                                               seed=SEED,
                                               target_size=(img_h, img_w)
                                               )



#valid_data_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
num_classes = 3


#TRAIN
train_dataset = tf.data.Dataset.from_generator(lambda: train_generator,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes])
                                               )
train_dataset = train_dataset.repeat()

#VALID
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_generator,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, img_h, img_w, 3], [None, num_classes])
                                               )
valid_dataset = valid_dataset.repeat()


In [ ]:
# Let's test data augmentation
# ----------------------------
import time
import matplotlib.pyplot as plt

%matplotlib inline
    
iterator = iter(train_dataset)

In [ ]:
augmented_img, target = next(iterator)
augmented_img = np.array(augmented_img[0])   # First element
augmented_img = augmented_img * 255  # denormalize
   
plt.imshow(np.uint8(augmented_img))
# fig.canvas.draw()
plt.plot()

In [ ]:
# Load VGG16 Model

vgg = tf.keras.applications.Xception(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))

In [ ]:
vgg.summary()

In [ ]:
# Create Model
# ------------

finetuning = True

if finetuning:
    freeze_until = 15 # layer from which we want to fine-tune
    
    for layer in vgg.layers[:freeze_until]:
        layer.trainable = False
else:
    vgg.trainable = False
    
model = tf.keras.Sequential()
model.add(vgg)
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(units=512, activation='relu'))
model.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

In [ ]:
# Visualize created model as a table
model.summary()

# Visualize initialized weights
#model.weights

In [ ]:
# Optimization params
# -------------------

# Loss
loss = tf.keras.losses.CategoricalCrossentropy()

# learning rate
lr = 2e-5
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
# -------------------

# Validation metrics
# ------------------

metrics = ['accuracy']
# ------------------

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
#%load_ext tensorboard

#%reload_ext tensorboard
#%tensorboard --logdir /content/drive/My\ Drive/DeepLearning/kaggle/tensorboard #fix here

In [ ]:
import os
from datetime import datetime


cwd = os.getcwd()

exps_dir = os.path.join('/content/drive/My Drive/DeepLearning/kaggle', 'tensorboard')
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   save_weights_only=True)  # False to save the model directly
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=1)  # if 1 shows weights histograms
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    callbacks.append(es_callback)

In [ ]:
train_dataset

In [ ]:
model.fit(x=train_dataset,
          epochs=100,  #### set repeat in training dataset
          steps_per_epoch=len(train_generator),
          validation_data=valid_dataset,
          validation_steps=len(valid_generator),
          callbacks=callbacks
          )

# How to visualize Tensorboard

# 1. tensorboard --logdir EXPERIMENTS_DIR --port PORT     <- from terminal
# 2. localhost:PORT   <- in your browser

In [ ]:
# Let's visualize the activations of our network
from PIL import Image

test_iter = iter(valid_dataset)

# Get a test image
test_img = next(test_iter)[0]
test_img = test_img[0]

# Visualize the image
Image.fromarray(np.uint8(np.array(test_img)*255.))

In [ ]:
test_dir = os.path.join(cwd, 'MaskDataset/test')
image_filenames = next(os.walk(test_dir))[2]

results = {}
for image_name in image_filenames:

   img = Image.open(os.path.join(test_dir,image_name)).convert('RGB')
   img = img.resize((512, 512))
   img_array = np.array(img)
   img_array = np.expand_dims(img_array, 0) 

   out_softmax = model.predict(x=img_array / 255.)
   prediction = tf.argmax(out_softmax,1)  # predicted class
   print(prediction.numpy()[0])

   results[image_name] = prediction.numpy()[0]

In [ ]:
create_csv(results)